#### Candlesticks and Trend analysis


In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import pandas_ta as ta
import matplotlib.pyplot as plt

df = yf.download(tickers=['AAPL'],start='2020-01-01',end='2023-01-01')
print("The number of data points are", len(df))

#Check if any zero volumes are to be removed
columns = df.columns 
for col in columns:
    indexZeros = df[ df[col] == 0 ].index
    df.drop(indexZeros , inplace=True)

In [ ]:
df['MACD'],_,_= df.ta.macd()
df['ATR'] = df.ta.atr(length=10)
df['RSI'] = df.ta.rsi()
df.dropna(inplace=True)
df.reset_index(inplace=True)
df.head(5)

In [ ]:

# Shooting star Strategy
alpha= 1.5
beta= 1
gamma= 1
rsi_upper_limit_buy = 90
rsi_lower_limit_buy = 50

rsi_upper_limit_sell = 40
rsi_lower_limit_sell = 10

def shooting_star_strategy(dataframe):
    dataframe.dropna()
    
    length = len(dataframe)
    high = list(dataframe['High'])
    low = list(dataframe['Low'])
    close = list(dataframe['Close'])
    open = list(dataframe['Open'])
    signal = [0] * length
    highdiff = [0] * length
    lowdiff = [0] * length
    bodydiff = [0] * length
    ratio1 = [0] * length
    ratio2 = [0] * length
        
    for row in range(0, length):
        
        highdiff[row] = high[row]-max(open[row],close[row])
        bodydiff[row] = abs(open[row]-close[row])
        if bodydiff[row]<0.01:
            bodydiff[row]=0.01
        lowdiff[row] = min(open[row],close[row])-low[row]
        ratio1[row] = highdiff[row]/bodydiff[row]
        ratio2[row] = lowdiff[row]/bodydiff[row]
    
        
        if (ratio1[row]>alpha and lowdiff[row]<beta*highdiff[row] and bodydiff[row]>0.05 and df.RSI[row]>rsi_lower_limit_buy and df.RSI[row]<rsi_upper_limit_buy ):
            signal[row] = 1
        
        elif (ratio2[row]>alpha and highdiff[row]<beta*lowdiff[row] and bodydiff[row]>0.05 and df.RSI[row]<rsi_upper_limit_sell and df.RSI[row]>rsi_lower_limit_sell):
            signal[row]=2

    return signal


df['signal'] = shooting_star_strategy(df)
print("Total number of Signals Generated using Shooting Star Strategy ", (df['signal']!=0).sum())
print("Ratio of Data points to signal is ",(df['signal']!=0).sum()/len(df))


In [ ]:
fig=plt.figure(figsize=[15,6])
plt.plot(df['Date'],df['Close'],c='orange',lw=0.2)
plt.scatter(x=df['Date'].loc[df['signal']==1],y=df['Close'].loc[df['signal']==1],marker="^",lw=0.1,c="green")
plt.scatter(x=df['Date'].loc[df['signal']==2],y=df['Close'].loc[df['signal']==2],marker="v",lw=0.1,c="red")


In [ ]:
#Creating Trends Signal to be used along side Stoooting Star Signal

def trend(look_ahead_period, dataframe):
    length = len(dataframe)
    high = list(dataframe['High'])
    low = list(dataframe['Low'])
    close = list(dataframe['Close'])
    open = list(dataframe['Open'])
    atr = list(dataframe['ATR'])
    category = [0] * length
    

    for line in range (0,length-look_ahead_period-1):
        OpenLow = 0
        OpenHigh = 0
        
        highdiff = high[line]-max(open[line],close[line])
        bodydiff = abs(open[line]-close[line])
        
        y_delta = atr[line]*1. # highdiff*1.3 #for SL 400*1e-3
        if y_delta<1.1:
            y_delta=1.1
            
        SLTPRatio = 2. #y_delta*Ratio gives TP
        
        for i in range(1,look_ahead_period+1):
            value1 = close[line]-low[line+i]
            value2 = close[line]-high[line+i]
            OpenLow = max(value1, OpenLow)
            OpenHigh = min(value2,OpenHigh)

            if ( (OpenLow >= (SLTPRatio*y_delta) ) and (-OpenHigh < y_delta) ):
                category[line] = 1 #-1 downtrend
                break
            elif ((OpenLow < y_delta) ) and (-OpenHigh >= (SLTPRatio*y_delta)):
                category[line] = 2 # uptrend
                break 
            else:
                category[line] = 0 # no clear trend
            
    return category

# trend(barsfront to take into account, dataframe)
df['Trend'] = trend(100, df)


In [ ]:
df['Trend'].value_counts()

In [ ]:
df.head(2)

In [ ]:
fig=plt.figure(figsize=[15,6])
plt.plot(df['Date'],df['Close'],lw=0.3)
plt.scatter(df['Date'].loc[df['Trend']==1],df['Close'].loc[df['Trend']==1],c='r',label='Downtrend')
plt.scatter(df['Date'].loc[df['Trend']==2],df['Close'].loc[df['Trend']==2],c='g',label='Uptrend')
plt.ylabel('Prices')
plt.xlabel("Dates")
plt.title('Trends Detection')

In [ ]:
df.head(2)

In [ ]:
df=df[['Date','Adj Close','signal','Trend']]

In [ ]:
df['Ret']=df['Adj Close'].pct_change()
df=df.dropna()

In [ ]:
df['Positions'] = 0
df['Positions'].loc[df['Trend'] ==1] = -1
df['Positions'].loc[df['Trend'] ==2] = 1

df['Strat_ret'] = df['Positions'].diff().shift(1) * df['Ret']
df['Positions_L'] = df['Positions'].diff().shift(1)
df.loc[df['Positions_L'] == -1, 'Positions_L'] = 0
df['Strat_ret_L'] = df['Positions_L'] * df['Ret']

df['CumRet'] = (1 + df['Strat_ret']).cumprod() - 1
df['CumRet_L'] = (1 + df['Strat_ret_L']).cumprod() - 1
df['bhRet'] = df['Ret'].cumsum()

Final_Return_L = np.prod(1 + df["Strat_ret_L"]) - 1
Final_Return = np.prod(1 + df["Strat_ret"]) - 1
Buy_Return = np.prod(1 + df["Ret"]) - 1

print("Strat Return Long Only =", Final_Return_L * 100, "%")
print("Strat Return =", Final_Return * 100, "%")
print("Buy and Hold Return =", Buy_Return * 100, "%")

fig = plt.figure(figsize=(12, 6))
ax = plt.gca()
df.plot(x="Date", y="bhRet", label=f"Buy & Hold Only {Buy_Return.round(3)*100} % Returns ", ax=ax)
df.plot(x="Date", y="CumRet_L", label=f"Taking Only Long from Strategy {Final_Return_L.round(3)*100} % Returns ", ax=ax)
df.plot(x="Date", y="CumRet", label=f"Taking Both Long Short from Strategy {Final_Return.round(3)*100} % Returns ", ax=ax)
plt.xlabel("date")
plt.ylabel("Cumulative Returns")
plt.grid()
plt.show()